In [5]:
import pandas as pd

# Load CSV files into DataFrames
df_lang = pd.read_csv('/content/HCK_HEC_LANG.csv')
df_skills = pd.read_csv('/content/HCK_HEC_SKILLS.csv')
df_staffing = pd.read_csv('/content/HCK_HEC_STAFFING.csv')
df_user = pd.read_csv('/content/HCK_HEC_USER.csv')
df_xp = pd.read_csv('/content/HCK_HEC_XP.csv')

# Display the first few rows of each DataFrame
print("HCK_HEC_LANG:")
print(df_lang.head())

print("\nHCK_HEC_SKILLS:")
print(df_skills.head())

print("\nHCK_HEC_STAFFING:")
print(df_staffing.head())

print("\nHCK_HEC_USER:")
print(df_user.head())

print("\nHCK_HEC_XP:")
print(df_xp.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/HCK_HEC_LANG.csv'

In [ ]:
# Get column names for each DataFrame
print("HCK_HEC_LANG Columns:")
print(df_lang.columns.tolist())

print("\nHCK_HEC_SKILLS Columns:")
print(df_skills.columns.tolist())

print("\nHCK_HEC_STAFFING Columns:")
print(df_staffing.columns.tolist())

print("\nHCK_HEC_USER Columns:")
print(df_user.columns.tolist())

print("\nHCK_HEC_XP Columns:")
print(df_xp.columns.tolist())


In [6]:
# Calculate the number of unique USER_IDs in each table
unique_user_lang = df_lang['USER_ID'].nunique()
unique_user_skills = df_skills['USER_ID'].nunique()
unique_user_staffing = df_staffing['USER_ID'].nunique()
unique_user_user = df_user['USER_ID'].nunique()
unique_user_xp = df_xp['USER_ID'].nunique()

# Calculate the total number of unique USER_IDs across all tables
total_unique_users = df_user['USER_ID'].nunique()  # Assuming df_user has all unique user ids

# Create a dictionary to store the results
results = {
    "HCK_HEC_LANG": {"Unique Users": unique_user_lang, "Percentage": (unique_user_lang / total_unique_users) * 100},
    "HCK_HEC_SKILLS": {"Unique Users": unique_user_skills, "Percentage": (unique_user_skills / total_unique_users) * 100},
    "HCK_HEC_STAFFING": {"Unique Users": unique_user_staffing, "Percentage": (unique_user_staffing / total_unique_users) * 100},
    "HCK_HEC_USER": {"Unique Users": unique_user_user, "Percentage": (unique_user_user / total_unique_users) * 100},
    "HCK_HEC_XP": {"Unique Users": unique_user_xp, "Percentage": (unique_user_xp / total_unique_users) * 100},
}

# Print the results
for table_name, data in results.items():
    print(f"{table_name}:")
    print(f"  Unique Users: {data['Unique Users']}")
    print(f"  Percentage of Total: {data['Percentage']:.2f}%")


NameError: name 'df_lang' is not defined

We can see that in some tables the USER_ID is not always unique, suggesting each user can have one or more row entries.


In [7]:
# Set the display options to show all columns
pd.set_option('display.max_rows', None)


In [8]:
df_skills['DOMAIN_DSC'].value_counts()

NameError: name 'df_skills' is not defined

In [ ]:
df_skills['SKILLS_DSC'].value_counts()

In [9]:
df_xp['MISSION_DSC'].value_counts()

NameError: name 'df_xp' is not defined

In [10]:
# Assuming you have your DataFrames (df_lang, df_skills, etc.) loaded as before

def null_counts(df, table_name):
  """Calculates null counts for each column in a DataFrame."""
  null_counts = df.isnull().sum()
  print(f"\nNull counts for {table_name}:")
  print(null_counts)

# Get null counts for each table
null_counts(df_lang, "HCK_HEC_LANG")
null_counts(df_skills, "HCK_HEC_SKILLS")
null_counts(df_staffing, "HCK_HEC_STAFFING")
null_counts(df_user, "HCK_HEC_USER")
null_counts(df_xp, "HCK_HEC_XP")


NameError: name 'df_lang' is not defined

In [11]:
# Find non-unique rows in df_staffing based on USER_ID
duplicate_user_ids = df_staffing[df_staffing.duplicated(subset=['USER_ID'], keep=False)]['USER_ID'].unique()

# Filter the DataFrame to include only rows with duplicate USER_IDs
df_duplicates = df_staffing[df_staffing['USER_ID'].isin(duplicate_user_ids)]

# Check if df_duplicates is empty before proceeding
if df_duplicates.empty:
    print("No duplicate USER_IDs found in df_staffing.")
    df_inconsistent_months = []  # Initialize as an empty list to avoid errors later
else:
    # Group by USER_ID and check if all values for month columns are the same
    def has_varying_months(group):
        month_cols = [f'MONTH_{i}' for i in range(1, 13)]
        return not group[month_cols].nunique().eq(1).all()

    # Apply the function to each group and filter for cases where months are not the same
    df_inconsistent_months = df_duplicates.groupby('USER_ID').apply(has_varying_months).loc[lambda x: x].index.tolist()

# Filter the DataFrame to show only rows with inconsistent months for the duplicate USER_IDs
df_inconsistent_staffing = df_staffing[df_staffing['USER_ID'].isin(df_inconsistent_months)]

print("Number of rows with non unique USER_IDs:", df_duplicates.shape[0])
print("Number of rows with non unique USER_IDs and inconsistent month columns:", df_inconsistent_staffing.shape[0])


NameError: name 'df_staffing' is not defined

In [12]:
# Define columns that need to be merged as comma-separated values
merge_columns = {
    "HCK_HEC_LANG": ["LANGUAGE_SKILL_DSC", "LANGUAGE_SKILL_LVL"],
    "HCK_HEC_SKILLS": ["SKILLS_DSC", "DOMAIN_DSC", "LEVEL_VAL"],
    "HCK_HEC_XP": ["MISSION_DSC"]
}

# Aggregate function for merging rows with comma separation
def merge_values(series):
    return ', '.join(series.dropna().astype(str).unique())

# Apply aggregation to each dataset
df_lang = df_lang.groupby("USER_ID").agg({col: merge_values for col in merge_columns["HCK_HEC_LANG"]}).reset_index()
df_skills = df_skills.groupby("USER_ID").agg({col: merge_values for col in merge_columns["HCK_HEC_SKILLS"]}).reset_index()
df_xp = df_xp.groupby("USER_ID").agg({col: merge_values for col in merge_columns["HCK_HEC_XP"]}).reset_index()

# Merge all tables on USER_ID
df_merged = df_user.copy()  # Start with user table (full reference)
df_merged = df_merged.merge(df_lang, on="USER_ID", how="left")
df_merged = df_merged.merge(df_skills, on="USER_ID", how="left")
df_merged = df_merged.merge(df_staffing, on="USER_ID", how="left")
df_merged = df_merged.merge(df_xp, on="USER_ID", how="left")

NameError: name 'df_lang' is not defined

In [13]:
df_merged.head()

NameError: name 'df_merged' is not defined

In [14]:

# Calculate null counts and percentages for each column in df_merged
null_counts = df_merged.isnull().sum()
null_percentages = (null_counts / len(df_merged)) * 100

# Create a DataFrame to display the results
null_analysis = pd.DataFrame({
    'Null Counts': null_counts,
    'Null Percentages': null_percentages
})

# Display the null analysis
null_analysis


NameError: name 'df_merged' is not defined

In [15]:
# prompt: df merged give me one row with missing values

# Find a row with missing values
row_with_missing_values = df_merged[df_merged.isnull().any(axis=1)].head(1)

row_with_missing_values


NameError: name 'df_merged' is not defined

In [16]:
# prompt: check if userid 2817191 is in df skills

if 2817191 in df_skills['USER_ID'].values:
  print("User ID 2817191 is present in df_skills.")
else:
  print("User ID 2817191 is not present in df_skills.")


NameError: name 'df_skills' is not defined

ROWS Are NAN because some user_ids don't have matching rows in other tables like skills

In [17]:
def split_numeric_column(df, col_name, prefix):
    # Split string values into lists of integers
    # Filter out empty strings before conversion
    df[col_name] = df[col_name].apply(lambda x: [int(float(i)) for i in x.split(", ") if i] if pd.notna(x) else [])

    # Expand into separate columns
    max_length = df[col_name].apply(len).max()  # Find max number of values
    for i in range(max_length):
        df[f"{prefix}_{i+1}"] = df[col_name].apply(lambda x: x[i] if i < len(x) else None)

    # Drop original column after expansion
    df.drop(columns=[col_name], inplace=True)

# Apply function to split `LANGUAGE_SKILL_LVL` and `LEVEL_VAL`
split_numeric_column(df_merged, "LANGUAGE_SKILL_LVL", "LANGUAGE_SKILL_LVL")
split_numeric_column(df_merged, "LEVEL_VAL", "LEVEL_VAL")

NameError: name 'df_merged' is not defined

In [ ]:
df_merged.head()

In [18]:
def match_language_skill(df):
    english_skills, french_skills = [], []

    for index, row in df.iterrows():
        # Check if LANGUAGE_SKILL_DSC is a string and not null before splitting
        if isinstance(row["LANGUAGE_SKILL_DSC"], str) and not pd.isnull(row["LANGUAGE_SKILL_DSC"]):
            languages = row["LANGUAGE_SKILL_DSC"].split(", ")  # Split into list
        else:
            languages = []  # Assign an empty list if not a string or null

        # Access the split columns directly using the prefix
        # Assuming skill levels are stored in columns like "LANGUAGE_SKILL_LVL_1", "LANGUAGE_SKILL_LVL_2", etc.
        # Get the number of LANGUAGE_SKILL_LVL_ columns
        num_skill_cols = len([col for col in df.columns if col.startswith("LANGUAGE_SKILL_LVL_")])

        skill_levels = [row[f"LANGUAGE_SKILL_LVL_{i+1}"] for i in range(num_skill_cols) if pd.notna(row[f"LANGUAGE_SKILL_LVL_{i+1}"])]
        skill_levels = [int(float(x)) for x in skill_levels]

        # Initialize default values
        english_level = None
        french_level = None

        # Assign skill levels based on language order
        for i, lang in enumerate(languages):
            if lang == "English":
                english_level = skill_levels[i] if i < len(skill_levels) else None  # Handle potential IndexError
            elif lang == "French":
                french_level = skill_levels[i] if i < len(skill_levels) else None  # Handle potential IndexError

        english_skills.append(english_level)
        french_skills.append(french_level)

    df["english_skill_lvl"] = english_skills
    df["french_skill_lvl"] = french_skills
    return df


# Apply function to match language skills
df_merged = match_language_skill(df_merged)

# Drop the original columns if no longer needed
df_merged.drop(columns=["LANGUAGE_SKILL_DSC"] + [col for col in df_merged.columns if col.startswith("LANGUAGE_SKILL_LVL_")], inplace=True)  # drop the split columns


NameError: name 'df_merged' is not defined

In [ ]:
df_merged.head()

In [19]:
# prompt: df merged to csv

# Assuming df_merged is your final DataFrame
df_merged.to_csv('merged_data.csv', index=False)


NameError: name 'df_merged' is not defined

In [ ]:
# prompt: skills_dsc has comma seperated value process them all like lower trim etc and then find how many unique ones are there and list them

import pandas as pd

# Assuming df_skills is your DataFrame
def process_skills(df_skills):
  """Processes skills data, removes duplicates, and returns unique skills."""

  unique_skills = set()
  if 'SKILLS_DSC' in df_skills.columns:
    for skills in df_skills['SKILLS_DSC'].dropna().astype(str):
      for skill in skills.split(', '):
          processed_skill = skill.strip().lower()
          unique_skills.add(processed_skill)

  return list(unique_skills)

unique_skill_list = process_skills(df_skills)
print("Number of Unique Skills:", len(unique_skill_list))
print("Unique Skills:")
for skill in unique_skill_list:
    print(skill) # Added indentation and print statement

Translate MISSION_DSC column into english

In [3]:
#pip install deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [4]:
from deep_translator import GoogleTranslator
import pandas as pd

# Load the dataset
df_merged = pd.read_csv('merged_data.csv')

# Initialize Google Translator
translator = GoogleTranslator(source='auto', target='en')

# Function to translate text safely
def safe_translate(text):
    try:
        if pd.notnull(text) and text.strip():  # Ensure text is not empty
            return translator.translate(text)
    except Exception as e:
        print(f"Translation error for '{text}': {e}")
        return text  # Return original text if translation fails
    return text

# Apply translation
df_merged['mission_dsc_eng'] = df_merged['MISSION_DSC'].astype(str).apply(safe_translate)

# Save the updated dataset
df_merged.to_csv('merged_data_translated.csv', index=False)

print("Translation completed and saved as 'merged_data_translated.csv'")


Translation error for 'Définir et déployer la vision, les objectifs et les plans d’action relatifs à l’Analytique en cohérence avec la stratégie globale de [CLIENT]., Collaborer étroitement avec le [CLIENT] pour stimuler la croissance des revenus et avec le [CLIENT] pour optimiser les opérations., Intervenir en tant que Customer Success Manager sur des comptes stratégiques pour assurer la bonne tenue de nos interventions, maintenir une excellente relation avec le client et générer des opportunités., S’assurer du bon taux d’activité des consultants de la BU et de sa rentabilité, Anticiper les risques et mettre en place des actions préventives et évolutives afin de les gérer, Représenter l’Entreprise sur son périmètre d’activité, faire connaître sa BU et son offre sur le marché., Assumer un rôle d'architecte, superviser les autres architectes et les faire profiter de son expérience pour leur permettre de monter en compétence., En lien avec le Bureau de projet, assurer la bonne tenue des 

Does not work well with very long descriptions, so translation error

In [5]:
import pandas as pd

# Load both datasets
df_original = pd.read_csv('merged_data.csv')
df_translated = pd.read_csv('merged_data_translated.csv')

# Compare shape (rows, columns)
print(f"Original Dataset: {df_original.shape[0]} rows, {df_original.shape[1]} columns")
print(f"Translated Dataset: {df_translated.shape[0]} rows, {df_translated.shape[1]} columns")

# Check if any rows were lost or columns were added/removed
if df_original.shape == df_translated.shape:
    print("✅ Both datasets have the same number of rows and columns.")
else:
    print("⚠️ There is a mismatch in rows or columns!")

# Check if an extra column 'mission_dsc_eng' was correctly added
if 'mission_dsc_eng' in df_translated.columns:
    print("✅ 'mission_dsc_eng' column is present in the translated dataset.")
else:
    print("❌ 'mission_dsc_eng' column is missing!")


Original Dataset: 119 rows, 30 columns
Translated Dataset: 119 rows, 31 columns
⚠️ There is a mismatch in rows or columns!
✅ 'mission_dsc_eng' column is present in the translated dataset.
